In [1]:
!git clone https://github.com/mikolajhojda/ship-detection.git

Cloning into 'ship-detection'...
remote: Enumerating objects: 258, done.
remote: Total 258 (delta 0), reused 0 (delta 0), pack-reused 258 (from 2)
Receiving objects: 100% (258/258), 266.34 MiB | 33.78 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (346/346), done.


In [2]:
import os
os.chdir('/content/ship-detection/src')

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [14]:
import os
import random
import shutil
from glob import glob
from ultralytics import YOLO

# === Paths ===
ROOT = os.getcwd()
DATA_DIR = os.path.abspath(os.path.join(ROOT, "../data"))
TILE_DIR = os.path.join(DATA_DIR, "tiles")
LABEL_DIR = os.path.join(DATA_DIR, "labels")

DATASET_DIR = os.path.abspath(os.path.join(ROOT, "../dataset"))
os.makedirs(DATASET_DIR, exist_ok=True)

# === Parameters ===
TRAIN_SPLIT = 0.8
IMG_SIZE = 512
EPOCHS = 50
MODEL_SIZE = "yolov8n.pt"


In [15]:
# === Make folders ===
for split in ["train", "val"]:
    os.makedirs(os.path.join(DATASET_DIR, f"images/{split}"), exist_ok=True)
    os.makedirs(os.path.join(DATASET_DIR, f"labels/{split}"), exist_ok=True)

# === Load and split ===
images = sorted(glob(os.path.join(TILE_DIR, "*.png")))
random.shuffle(images)
split_idx = int(len(images) * TRAIN_SPLIT)

train_imgs = images[:split_idx]
val_imgs = images[split_idx:]

def copy_split(img_list, split):
    for img_path in img_list:
        filename = os.path.basename(img_path)
        name = os.path.splitext(filename)[0]
        label_path = os.path.join(LABEL_DIR, f"{name}.txt")

        if os.path.exists(label_path):
            shutil.copy(img_path, os.path.join(DATASET_DIR, f"images/{split}/{filename}"))
            shutil.copy(label_path, os.path.join(DATASET_DIR, f"labels/{split}/{name}.txt"))

copy_split(train_imgs, "train")
copy_split(val_imgs, "val")


In [16]:
# === Create data.yaml ===
yaml_content = f"""
path: {DATASET_DIR}
train: images/train
val: images/val

names:
  0: ship
"""

with open(os.path.join(DATASET_DIR, "data.yaml"), "w") as f:
    f.write(yaml_content)

In [17]:
# === Train YOLOv8 ===
print("\n✅ Training YOLOv8...\n")
model = YOLO(MODEL_SIZE)
model.train(data=os.path.join(DATASET_DIR, "data.yaml"), epochs=EPOCHS, imgsz=IMG_SIZE, batch=50)


✅ Training YOLOv8...

Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/ship-detection/dataset/data.yaml, epochs=50, time=None, patience=100, batch=50, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

train: Scanning /content/ship-detection/dataset/labels/train.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1568.3±275.9 MB/s, size: 212.5 KB)



val: Scanning /content/ship-detection/dataset/labels/val... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<00:00, 589.31it/s]

val: New cache created: /content/ship-detection/dataset/labels/val.cache
Plotting labels to runs/detect/train3/labels.jpg... 


...

                   all         25         60    0.00613      0.767     0.0163     0.0103



50 epochs completed in 0.384 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.32s/it]


                   all         25         60    0.00533      0.667      0.105     0.0717
Speed: 1.8ms preprocess, 195.8ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x784c05988350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [18]:
# === Evaluate model ===
model.val()

Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1476.9±608.2 MB/s, size: 105.6 KB)


val: Scanning /content/ship-detection/dataset/labels/val.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:04<00:00,  4.66s/it]


                   all         25         60    0.00533      0.667      0.105     0.0717
Speed: 1.2ms preprocess, 166.6ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to runs/detect/train32


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x784c02e01e50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [33]:
model = YOLO("runs/detect/train3/weights/best.pt")
results = model.predict(source="../dataset/images/val", save=True, conf=0.013)


image 1/25 /content/ship-detection/src/../dataset/images/val/tile_0_suez_14.04.25_false.png: 512x512 (no detections), 289.3ms
image 2/25 /content/ship-detection/src/../dataset/images/val/tile_10_malacca_09.04.25_false.png: 512x512 (no detections), 274.4ms
image 3/25 /content/ship-detection/src/../dataset/images/val/tile_10_north_sea_18.07.24_false.png: 512x512 1 ship, 246.3ms
image 4/25 /content/ship-detection/src/../dataset/images/val/tile_11_malacca_09.04.25_false.png: 512x512 (no detections), 236.7ms
image 5/25 /content/ship-detection/src/../dataset/images/val/tile_11_north_sea_18.07.24_false.png: 512x512 (no detections), 268.8ms
image 6/25 /content/ship-detection/src/../dataset/images/val/tile_13_malacca_09.04.25_false.png: 512x512 (no detections), 250.4ms
image 7/25 /content/ship-detection/src/../dataset/images/val/tile_15_malacca_09.04.25_false.png: 512x512 (no detections), 250.3ms
image 8/25 /content/ship-detection/src/../dataset/images/val/tile_1_malacca_19.04.25_false.png: 51